# Exercise 1

Supervisor: Gregor Geigle
If you have questions, errors or problems:
1) Open a thread in the WueCampus forum: https://wuecampus.uni-wuerzburg.de/moodle/mod/forum/view.php?id=3093380
2) If your question/problem is personal and not appropriate for the forum, send a mail to gregor.geigle@uni-wuerzburg.de.

## 0 Setting Up

In this exercise, you will write your own deep learning model code. Before you can run the code here, you will have to install the dependencies.
You are given code skeletons and you will have to complete them according to the task. DO NOT CHANGE THE CODE SKELETONS.

The exercise requires:
0) Python Version 3.9 or greater (obviously)
1) [torch (PyTorch)](https://pytorch.org/get-started/locally/) -> PyTorch is a library for working with tensors on GPUs (or CPUs; you do not need a GPU for the exercises). This is **the** library for implementing and training deep learning models and is the main library for the exercises.
2) [Numpy](https://numpy.org/install/) -> Another library for working with tensors. It is CPU-only.
3) [scipy](https://scipy.org/install/) -> A library build on numpy for various scientific computing. You will not use this library yourself but some functions are needed for the code.

To install the requirements, you can run `pip install torch numpy scipy`. Once done, you can run the cell below and if it executes without error, you are ready.

In [1]:
import torch
import numpy
import scipy

torch.set_printoptions(precision=2) # limit precision when printing tensors to 2 digits
# This cell should run without problem once you install all requirements. If you get an ImportError, you forgot to install one of the libraries.

## 1 Layers and Activations and Initializations

In the first part of this exercise, you will implement different layers and activation functions in PyTorch yourself by completing the provided code skeletons.

IMPORTANT: PyTorch already implements all the layers and activation functions. You are (obviously) not allowed to use these functions. You are limited to basic operations (like `torch.matmul (Matrix multiplication)/ @ (the operation symbol for matrix multiplication), * (element-wise multiplication), torch.exp, torch.log, torch.sum)`. 

You can read more about those functions in the [official documentation](https://pytorch.org/docs/stable/index.html).

### 1.1 Matrix multiplication warmup

This exercise is to be done by hand. You are given three matrices `A=[[a,b,c],[d,e,f]], B=[[g,h],[i,j],[k,l]], C=[[m,n],[o,p]]`.
Compute the following or indicate if this operation is not allowed. 
1. A @ B [@ is the matrix multiplication]
2. B @ A
3. A @ C
4. B @ C
5. A * B.T  [* is the element-wise multiplication, .T indicates a transposed matrix)

**Fill in your answer and do not otherwise modify this cell**

1. 
$$
\begin{align*}
A @ B &= 
\begin{bmatrix}
a * g + b * i + c * k & a * h + b * j + c * l \\
d * g + e * i + f * k & d * h + e * j + f * l
\end{bmatrix}
\end{align*}
$$ 


2. 
$$
\begin{align*}
B @ A &= 
\begin{bmatrix}
g * a + h * d & g * b + h * e & g * c + h * f \\
i * a + j * d & i * b + j * e & i * c + j * f \\
k * a + l * d & k * b + l * e & k * c + l * f
\end{bmatrix}
\end{align*}
$$


3. 
$$
\begin{align*}
B @ C &= 
\begin{bmatrix}
g * m + h * o & g * n + h * p \\
i * m + j * o & i * n + j * p \\
k * m + l * o & k * n + l * p
\end{bmatrix}
\end{align*}
$$


4.

$$
\begin{align*}
A * B^T &= 
\begin{bmatrix}
a * g & b * i & c * k \\
d * h & e * j & f * l
\end{bmatrix}
\end{align*}
$$

### 1.2 Building up towards complex blocks

In this exercise, we will start with implementing fundamental layers and then combine those layers into more complex modules.


In [2]:
class Module:
    """
    This class is the base class for all layers and modules that you will implement.
    """        
    def forward(self, x):
        """ Return the result of the forward pass of this layer. Save intermediate results
        necessary to compute the gradients in the backward pass. 
        """
        raise NotImplementedError('Base class - method is not implemented')
    
def test_module(module, input=None):
    if input is None:
        input = torch.randn((2, 3))
    print(f"Input: {input}")
    result = module.forward(input)
    print(f"Output: {result}\n")

#### 1.2.1 Activation Functions
As you have seen in the lecture, there are many different activation functions. For this exercise, we will implement the ReLU activation from the lecture and also one function called **SiLU** or Sigmoid Linear Unit. We will first implement the Sigmoid and then use this for the SiLU module.

The definition for the functions is:
$$ \mathrm{ReLU}(x) = \max(0, x) $$
$$ \mathrm{Sigmoid}(x) = \frac{1}{1+\exp(-x)} $$
$$ \mathrm{SiLU}(x) = x * \mathrm{Sigmoid}(x) $$

(Hint for ReLU: `torch.clamp`)

In [5]:
class ReLU(Module):
    
    def forward(self, x):
        """ Return the element-wise ReLU of the input.
            param: x (torch.tensor): input to the activation function, of arbitrary shape
            returns (torch.tensor): element-wise ReLU(x), of the same shape as x
        """
        return torch.clamp(x, min=0)

class Sigmoid(Module):
    
    def forward(self, x):
        """ Return the element-wise sigmoid of the input.
            param: x (torch.tensor): input to the activation function, of arbitrary shape
            returns (torch.tensor): element-wise sigmoid(x), of the same shape as x
        """
        return 1/(1+torch.exp(-x))
        
        
class SiLU(Module):
    def __init__(self):
        self.sigmoid = Sigmoid()
        
    def forward(self, x):
        """ Return the result of a SiLU activation of the input.
            param: x (torch.tensor): input to the activation function, of arbitrary shape
            returns (torch.tensor): element-wise SiLU(x), of the same shape as x
        """
        return self.sigmoid.forward(x) * x

In [8]:
# You can manually test your implementation using the test_module function. Feel free to change this cell and re-run it for your tests.
test_module(ReLU()) # creates a random tensor as input
test_module(Sigmoid()) # creates a random tensor as input
test_module(Sigmoid(), input=torch.tensor([-100, -1, 0, 1, 100])) # test with a specific tensor

Input: tensor([[ 1.71,  1.02, -1.40],
        [-0.13, -0.90,  0.85]])
Output: tensor([[1.71, 1.02, 0.00],
        [0.00, 0.00, 0.85]])

Input: tensor([[ 1.50, -0.32,  0.09],
        [ 0.92, -0.37,  0.04]])
Output: tensor([[0.82, 0.42, 0.52],
        [0.72, 0.41, 0.51]])

Input: tensor([-100,   -1,    0,    1,  100])
Output: tensor([0.00, 0.27, 0.50, 0.73, 1.00])



In [9]:
# Automatic tests to verify your solution. DO NOT CHANGE THIS CELL
%run Tests/TestActivationFunctions.py
TestReLU.ReLU = ReLU
TestSigmoid.Sigmoid = Sigmoid
TestSiLU.SiLU = SiLU
unittest.main(argv=['first-arg-is-ignored'], exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 0.003s

OK


#### 1.2.2 Fully Connected Layer
Next, we are implementing the fully connected layer. This is your first parametrized module, that is, the first module that has parameters that would be optimized during training of a model.

Given a weight matrix $W \in \mathbb{R}^{n \times m}$ and a bias vector $b \in \mathbb{R}^{m}$, the output of the fully connected layer is $xW + b$.

It is important to properly *initialize* the parameters. For this exercise we will keep it simple and initialize $W$ with values from a normal distribution with mean 0 and standard deviation 0.02, and $b$ is initialized with all 0s. Hint: `torch.normal, torch.zeros`

In [10]:
class FullyConnectedLayer(Module):
    def __init__(self, input_size, output_size):
        """ A fully connected layer.
            param: input_size (int): dimension n of the input vector
            param: output_size (int): dimension m of the output vector
        """
        self.weights = self.init_weights(input_size, output_size, is_bias=False)
        self.bias = self.init_weights(output_size, is_bias=True)

    def forward(self, x):
        """ Compute the foward pass through the layer.
            param: x (torch.tensor): input with shape [b, n] where b is the batch size and n is the input size
            returns (torch.tensor): result of the forward pass, of shape [b, m] where b is the batch size and
                   m is the output size
        """
        return x @ self.weights + self.bias
    
    def init_weights(self, *dimensions, is_bias=False):
        """ Create a tensor of weights. The bias is initialized with all zeros. The weights are initialized with values from a normal distribution with mean 0 and standard deviation 0.02
            param: dimensions (list of ints): the dimensions of the tensor
            returns (torch.tensor): the tensor of weights
        """
        return torch.zeros(dimensions) if is_bias else torch.normal(0, 0.02, dimensions)
    

In [11]:
#Feel free to change this cell and re-run it for your tests.
fc = FullyConnectedLayer(4, 3)
print(fc.bias)
print(fc.weights)
test_module(fc, input=torch.randn((2, 4)))

tensor([0., 0., 0.])
tensor([[-0.01,  0.00,  0.02],
        [-0.00, -0.01, -0.04],
        [ 0.00, -0.00, -0.00],
        [ 0.03, -0.07, -0.03]])
Input: tensor([[-1.82, -1.64,  0.20, -1.07],
        [-0.36,  0.12, -0.42, -0.44]])
Output: tensor([[-0.01,  0.08,  0.05],
        [-0.01,  0.03,  0.00]])



In [12]:
# Automatic tests to verify your solution. DO NOT CHANGE THIS CELL
%run Tests/TestFullyConnected.py
TestFullyConnected.FullyConnected = FullyConnectedLayer
unittest.main(argv=['first-arg-is-ignored'], exit=False)

........
----------------------------------------------------------------------
Ran 8 tests in 0.080s

OK


#### 1.2.3 Putting it all together

More complex deep learning architectures often contain *blocks* made of different layers that are repeated multiple times.
To reduce code repetition, we create modules that encapsulate those blocks.

Here, we will create a block for a 2-layer fully connected network with an activation function (SiLU) inbetween.
Do not implement this from scratch but use your above implementations for SiLU and fully connected layers.


In [13]:
class SiLUBlock(Module):
    def __init__(self, input_size, hidden_size, output_size):
        """ A fully connected layer.
            param: input_size (int): dimension n of the input vector
            param: hidden_size (int): dimension h of the hidden vector
            param: output_size (int): dimension m of the output vector
        """
        # TODO: define the necessary class variables
        self.fully_connected_in = FullyConnectedLayer(input_size, hidden_size)
        self.fully_connected_out = FullyConnectedLayer(hidden_size, output_size)
        self.activation = SiLU()
        pass
    
    def forward(self, x):
        """ Compute the foward pass through the layer.
            param: x (torch.tensor): input with shape [b, n] where b is the batch size and n is the input size
            returns (torch.tensor): result of the forward pass, of shape [b, m] where b is the batch size and
                   m is the output size
        """
        return self.fully_connected_out.forward(self.activation.forward(self.fully_connected_in.forward(x)))
    
        

In [14]:
# Automatic tests to verify your solution. DO NOT CHANGE THIS CELL
%run Tests/TestBlocks.py
TestSiLUBlock.SiLUBlock = SiLUBlock
unittest.main(argv=['first-arg-is-ignored'], exit=False)

..........
----------------------------------------------------------------------
Ran 10 tests in 0.073s

OK


#### 1.2.4 BONUS: The SiGLU Blocks (not graded)

This is an ungraded bonus exercise that combines all previously learned in one block. 
The SiGLU (or SwiGLU; for SiLU or Swish Gated Linear Unit) block that you implement in this exercise is a very popular block in the real world as it is used in the majority of large language models.

Its implementation is similar to the SiLU block before but we add an additional gating part:

Given an input matrix $I \in \mathbb{R}^{n \times h}$, a gate matrix $G \in \mathbb{R}^{n \times h}$, and an output matrix $O \in \mathbb{R}^{h \times m}$ (together with matching biases), the SiGLU block is defined as:
$$ (xI * \mathrm{SiLU}(xG))O $$ 

 


In [19]:
class SiGLUBlock(Module):
    def __init__(self, input_size, hidden_size, output_size):
        """ A fully connected layer.
            param: input_size (int): dimension n of the input vector
            param: hidden_size (int): dimension h of the hidden vector
            param: output_size (int): dimension m of the output vector
        """
        # TODO: define the necessary class variables
        self.fully_connected_in = FullyConnectedLayer(input_size, hidden_size)
        self.fully_connected_gate = FullyConnectedLayer(input_size, hidden_size)
        self.fully_connected_out = FullyConnectedLayer(hidden_size, output_size)
        self.activation = SiLU()
        pass
    
    def forward(self, x):
        """ Compute the foward pass through the layer.
            param: x (torch.tensor): input with shape [b, n] where b is the batch size and n is the input size
            returns (torch.tensor): result of the forward pass, of shape [b, m] where b is the batch size and
                   m is the output size
        """
        return self.fully_connected_out.forward(
            self.fully_connected_in.forward(x) * self.activation.forward(self.fully_connected_gate.forward(x))
        )

In [20]:
# Automatic tests to verify your solution. DO NOT CHANGE THIS CELL
%run Tests/TestBlocks2.py
TestSiGLUBlock.SiGLUBlock = SiGLUBlock
unittest.main(argv=['first-arg-is-ignored'], exit=False)

............
----------------------------------------------------------------------
Ran 12 tests in 0.077s

OK


In practice, we want the SiGLUBlock to use the same number of parameters as the SiLUBlock. Assuming we have input and output dimension $h$ and the SiLUBlock has hidden dimension $2h$, what is the hidden dimension of the SiGLUBlock with equivalent number of parameters?


**Fill in your answer and do not otherwise modify this cell**
 
The number of parameters in SiLu block is $4h^2 + 3h$. At the same time the number of parameters in SwiGLU block is $3h x$, where $x$ is the hidden output size.

$$
\begin{align*}
3h x &= 4h^2 + 3h \\
x &= \frac{4}{3}h + 1
\end{align*}
$$

## 2 The Softmax Activation

In the second part of the exercise, we will look at the softmax activation function in some more detail.

The softmax function is a useful function because it can turn any arbitrary vector into a probability distribution (i.e., all terms are $\geq 0$ and add up to 1).
With the softmax, you can, for example, turn a model's output for a classification task with 3 labels into a distribution over those 3 classes.

### 2.1 Implementation of Softmax

Implement the softmax module.
Recall the definition $\mathrm{softmax}(x) = \frac{\exp x}{\sum_{x_i \in x} \exp x_i}$

Your implementation has to be written **fully** with PyTorch functions, in other words, you are not allowed to use Python loops to sum up the values!

In [21]:
class Softmax(Module):
    
    def forward(self, x):
        """ Compute the forward pass through the layer.
            param: x (torch.tensor): input with shape [b, n] where b is the batch size and n is the input size
            returns (torch.tensor): result of the forward pass, of shape [b, n] where b is the batch size and n is the input size
        """
        return torch.exp(x) / torch.sum(torch.exp(x), dim=1).view(-1, 1)

In [22]:
# Automatic tests to verify your solution. DO NOT CHANGE THIS CELL
%run Tests/TestSoftmax.py
TestSoftmax.Softmax = Softmax
unittest.main(argv=['first-arg-is-ignored'], exit=False)

..............
----------------------------------------------------------------------
Ran 14 tests in 0.118s

OK


### 2.2 More Stable Implementation

The previous "naive" implementation of the softmax is not numerically stable because large values can cause overflow with the exponent (note the `nan` or "Not a Number" in the following output):

In [23]:
unstable_implementation = Softmax()
tensor = torch.tensor([[0.1, 0.2, 1000]])

print(unstable_implementation.forward(tensor))

tensor([[0., 0., nan]])


Implement a more stable version of softmax. Use the fact that $\mathrm{softmax}(x)=\mathrm{softmax}(x-b)$ with a good choice of $b$ to prevent overflow.

Note: You are **strongly** encouraged to derive on paper why this equality is correct. 

$$
\begin{align*}
Softmax(x - b) &= \frac{e^{x-b}}{\sum_i^M e^{x-b}} = \frac{e^{x} \times e^{-b}}{e^{-b} \times \sum_i^M e^{x}} = \frac{e^{x}}{\sum_i^M e^{x}} = Softmax(x)
\end{align*}
$$

In [24]:
class StableSoftmax(Softmax):
    
    def forward(self, x):
        """ Compute the forward pass through the layer.
            param: x (torch.tensor): input with shape [b, n] where b is the batch size and n is the input size
            returns (torch.tensor): result of the forward pass, of shape [b, n] where b is the batch size and n is the input size
        """
        return torch.exp(x - torch.max(x, dim=1).values.view(-1, 1)) / torch.sum(torch.exp(x - torch.max(x, dim=1).values.view(-1, 1)), dim=1).view(-1, 1)
    

In [25]:
stable_implementation = StableSoftmax()
tensor = torch.tensor([[0.1, 0.2, 1000]])

print(stable_implementation.forward(tensor))

tensor([[0., 0., 1.]])


In [26]:
# Automatic tests to verify your solution. DO NOT CHANGE THIS CELL
%run Tests/TestSoftmaxStable.py
TestStableSoftmax.StableSoftmax = StableSoftmax
unittest.main(argv=['first-arg-is-ignored'], exit=False)

.................
----------------------------------------------------------------------
Ran 17 tests in 0.312s

OK
